In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.feature_squeezing import FeatureSqueezing

feature_squeezer = FeatureSqueezing(clip_values=(0, 1), bit_depth=1)

def preprocess(inputs, feature_squeezer):
    inputs_squeezed, _ = feature_squeezer(inputs)
    return torch.tensor(inputs_squeezed, dtype=torch.float32)


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./FeatureSqueezing.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor = preprocess(X_test, feature_squeezer).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./FeatureSqueezing.csv", mode='a', index=False, header=False)




In [5]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     X_test_tensor = preprocess(X_test, feature_squeezer).to(device)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def5/y_pred_{attack_name}{eps}_Def5.npy", all_preds)


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [9]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor = preprocess(x_train, feature_squeezer).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor = preprocess(x_val, feature_squeezer).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [10]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# for epoch in range(50):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [12]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/preprocessor/dnn_pytorch_FeatureSqueezing.pt"))

/tmp/ipykernel_176/2979463627.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/pre

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./FeatureSqueezing.csv", mode='a', index=False, header=False)

In [13]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(66, 56) (66,)
Accuracy: 0.9090909090909091

macro
Precision: 0.90625
Recall: 0.925
F1 Score: 0.907735321528425

weighted
Precision: 0.9261363636363636
Recall: 0.9090909090909091
F1 Score: 0.9101075997627722

start XGB 100 BIM
(451, 56) (451,)
Accuracy: 0.6452328159645233

macro
Precision: 0.5771048894299047
Recall: 0.7122235872235871
F1 Score: 0.5331262939958592

weighted
Precision: 0.8901483999623928
Recall: 0.6452328159645233
F1 Score: 0.7172650608493661

start XGB 100 FGSM
(114, 56) (114,)
Accuracy: 0.32456140350877194

macro
Precision: 0.5470588235294118
Recall: 0.6367924528301887
F1 Score: 0.3008363201911589

weighted
Precision: 0.9364293085655314
Recall: 0.32456140350877194
F1 Score: 0.41155337567335304

start XGB 100 PGD
(451, 56) (451,)
Accuracy: 0.6452328159645233

macro
Precision: 0.5771048894299047
Recall: 0.7122235872235871
F1 Score: 0.5331262939958592

weighted
Precision: 0.8901483999623928
Recall: 0.6452328159645233
F1 Score: 0.7172650608493661

st

In [12]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(72, 56) (72,)
Accuracy: 0.9583333333333334

macro
Precision: 0.9558823529411764
Recall: 0.9634146341463414
F1 Score: 0.957935735150925

weighted
Precision: 0.9620098039215687
Recall: 0.9583333333333334
F1 Score: 0.9585037325543655

start XGB 100 BIM
(824, 56) (824,)
Accuracy: 0.5885922330097088

macro
Precision: 0.5242125228080284
Recall: 0.763746322327558
F1 Score: 0.41484996072270225

weighted
Precision: 0.9760201319019636
Recall: 0.5885922330097088
F1 Score: 0.7189956756839206

start XGB 100 FGSM
(288, 56) (288,)
Accuracy: 0.1597222222222222

macro
Precision: 0.4911290322580645
Recall: 0.46834532374100724
F1 Score: 0.15050460728389645

weighted
Precision: 0.9181339605734765
Recall: 0.1597222222222222
F1 Score: 0.23284863406627337

start XGB 100 PGD
(824, 56) (824,)
Accuracy: 0.5885922330097088

macro
Precision: 0.5242125228080284
Recall: 0.763746322327558
F1 Score: 0.41484996072270225

weighted
Precision: 0.9760201319019636
Recall: 0.5885922330097088
F1 Score

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.3012345679012346

macro
Precision: 0.4946455593062507
Recall: 0.4339554917444365
F1 Score: 0.24110601275251772

weighted
Precision: 0.9585946245784059
Recall: 0.3012345679012346
F1 Score: 0.4473364080432921

start XGB 100 VNIFGSM
(927, 56) (927,)
Accuracy: 0.2319309600862999

macro
Precision: 0.5018231478320245
Recall: 0.5140334279407127
F1 Score: 0.20148856476418153

weighted
Precision: 0.9585240773826295
Recall: 0.2319309600862999
F1 Score: 0.350336740786694

start XGB 50 baseline
(67, 56) (67,)
Accuracy: 0.9402985074626866

macro
Precision: 0.9411764705882353
Recall: 0.9459459459459459
F1 Score: 0.9401785714285714

weighted
Precision: 0.9473222124670764
Recall: 0.9402985074626866
F1 Score: 0.9404584221748401

start XGB 50 BIM
(881, 56) (881,)
Accuracy: 0.5993189557321226

macro
Precision: 0.5260953437978966
Recall: 0.7067283537095963
F1 Score: 0.42815034651926953

weighted
Precision: 0.9606959182276555
Recall: 0.5993189557321226
F1 Score: 0.7211256631180325

start XGB 50

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5429292929292929

macro
Precision: 0.4568910256410257
Recall: 0.389110271384404
F1 Score: 0.3885082967197031

weighted
Precision: 0.7785102628852628
Recall: 0.5429292929292929
F1 Score: 0.6321675570705132

start XGB 1 FGSM
(259, 56) (259,)
Accuracy: 0.10424710424710425

macro
Precision: 0.43506493506493504
Recall: 0.26190476190476186
F1 Score: 0.09831932773109243

weighted
Precision: 0.8343278343278343
Recall: 0.10424710424710425
F1 Score: 0.1674767204178969

start XGB 1 PGD
(396, 56) (396,)
Accuracy: 0.5429292929292929

macro
Precision: 0.4568910256410257
Recall: 0.389110271384404
F1 Score: 0.3885082967197031

weighted
Precision: 0.7785102628852628
Recall: 0.5429292929292929
F1 Score: 0.6321675570705132

start XGB 1 DF
(13, 56) (13,)
Accuracy: 0.8461538461538461

macro
Precision: 0.6666666666666666
Recall: 0.9166666666666667
F1 Score: 0.7045454545454546

weighted
Precision: 0.9487179487179488
Recall: 0.8461538461538461
F1 Score: 0.8776223776223776

start XGB 1 AutoPGD
(370

...

Accuracy: 0.7630331753554502

macro
Precision: 0.7389140841521794
Recall: 0.6935599101857522
F1 Score: 0.7058987511150758

weighted
Precision: 0.7544954439560623
Recall: 0.7630331753554502
F1 Score: 0.750746202400531

start RF 50 AutoPGD
(333, 56) (333,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 ZOO
(352, 56) (352,)
Accuracy: 0.9488636363636364

macro
Precision: 0.9617021276595745
Recall: 0.9333333333333333
F1 Score: 0.9443742098609356

weighted
Precision: 0.9527804642166344
Recall: 0.9488636363636364
F1 Score: 0.9480555395931503

start RF 50 CaFA
(23, 56) (23,)
Accuracy: 0.391304347826087

macro
Precision: 0.5625
Recall: 0.6666666666666666
F1 Score: 0.3611111111111111

weighted
Precision: 0.9239130434782609
Recall: 0.391304347826087
F1 Score: 0.4758454106280193

start RF 50 SINIFGSM
(7, 56) (7,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start RF 20 DF
(68, 56) (68,)
Accuracy: 0.8088235294117647

macro
Precision: 0.8049242424242424
Recall: 0.7908762420957542
F1 Score: 0.7960784313725491

weighted
Precision: 0.807653743315508
Recall: 0.8088235294117647
F1 Score: 0.8065743944636677

start RF 20 AutoPGD
(165, 56) (165,)
Accuracy: 0.9636363636363636

macro
Precision: 0.9633333333333334
Recall: 0.9633333333333334
F1 Score: 0.9633333333333334

weighted
Precision: 0.9636363636363636
Recall: 0.9636363636363636
F1 Score: 0.9636363636363636

start RF 20 ZOO
(195, 56) (195,)
Accuracy: 0.8769230769230769

macro
Precision: 0.8893762183235867
Recall: 0.8783143939393939
F1 Score: 0.8761904761904762

weighted
Precision: 0.8905083220872694
Recall: 0.8769230769230769
F1 Score: 0.876043956043956

start RF 20 CaFA
(8, 56) (8,)
Accuracy: 0.75

macro
Precision: 0.5
Recall: 0.375
F1 Score: 0.428571

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.9278350515463918

macro
Precision: 0.9539473684210527
Recall: 0.875
F1 Score: 0.9044334975369458

weighted
Precision: 0.9344818231144872
Recall: 0.9278350515463918
F1 Score: 0.9244223249200142

start DT 100 BIM
(3414, 56) (3414,)
Accuracy: 0.26362038664323373

macro
Precision: 0.36114303764709466
Recall: 0.3634580911713964
F1 Score: 0.2635951140609572

weighted
Precision: 0.45650092392278996
Recall: 0.26362038664323373
F1 Score: 0.26136859956239433

start DT 100 FGSM
(837, 56) (837,)
Accuracy: 0.43727598566308246

macro
Precision: 0.4412461738772899
Recall: 0.40721049783549784
F1 Score: 0.38311908804268735

weighted
Precision: 0.6242357579321278
Recall: 0.43727598566308246
F1 Score: 0.49383500374244665

start DT 100 PGD
(3414, 56) (3414,)
Accuracy: 0.26362038664323373

macro
Precision: 0.36114303764709466
Recall: 0.3634580911713964
F1 Score: 0.2635951140609572

weighted
Precision: 0.45650092392278996
Recall: 0.26362038664323373
F1 Score: 0.26136859956239433

start DT 100 DF

In [12]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(34, 56) (34,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(756, 56) (756,)
Accuracy: 0.626984126984127

macro
Precision: 0.5382356148449711
Recall: 0.7877975376196991
F1 Score: 0.45343793262574983

weighted
Precision: 0.967389379024725
Recall: 0.626984126984127
F1 Score: 0.7410520481586471

start XGB 100 FGSM
(265, 56) (265,)
Accuracy: 0.20754716981132076

macro
Precision: 0.5212870209059234
Recall: 0.5490712851405622
F1 Score: 0.20043103448275862

weighted
Precision: 0.9207481427914009
Recall: 0.20754716981132076
F1 Score: 0.2667534157449577

start XGB 100 PGD
(756, 56) (756,)
Accuracy: 0.626984126984127

macro
Precision: 0.5382356148449711
Recall: 0.7877975376196991
F1 Score: 0.45343793262574983

weighted
Precision: 0.967389379024725
Recall: 0.626984126984127
F1 Score: 0.7410520481586471

start XGB 100 DF
(219, 56) (219,)
Accuracy: 0.8721461187214612

macro
Precision: 0.896

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 100 ZOO
(270, 56) (270,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 100 CaFA
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input100/x_test_adv_CaFA_Def5.npy not found
start RF 100 SINIFGSM
(9, 56) (9,)
Accuracy: 0.8888888888888888

macro
Precision: 0.5
Recall: 0.4444444444444444
F1 Score: 0.47058823529411764

weighted
Precision: 1.0
Recall: 0.8888888888888888
F1 Score: 0.9411764705882353

start RF 100 VNIFGSM
(3, 56) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 baseline
(52, 56) (52,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 BIM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7699115044247787

macro
Precision: 0.8229004463822278
Recall: 0.7463762376237624
F1 Score: 0.7470076638250237

weighted
Precision: 0.8115456731056315
Recall: 0.7699115044247787
F1 Score: 0.7550913722719961

start RF 50 AutoPGD
(156, 56) (156,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 ZOO
(156, 56) (156,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 CaFA
(3, 56) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 SINIFGSM
(19, 56) (19,)
Accuracy: 0.9473684210526315

macro
Precision: 0.5
Recall: 0.47368421052631576
F1 Score: 0.4864864864864865

weighted
Precision: 1.0
Recall: 0.9473684210526315
F1 Score: 0.972972972972973

start RF 50 VNIFGSM
(6, 56) (6,)
Accuracy: 0.8333333333333334

macro
Precision: 0.5
Recall: 0.4166666666666667
F1 

...

Accuracy: 0.224

macro
Precision: 0.2566518847006652
Recall: 0.2954333643988817
F1 Score: 0.21964945902846805

weighted
Precision: 0.28011529933481155
Recall: 0.224
F1 Score: 0.1974306247810013

start RF 100 baseline
(90, 56) (90,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 100 BIM
(225, 56) (225,)
Accuracy: 0.6444444444444445

macro
Precision: 0.601510475881111
Recall: 0.6428571428571428
F1 Score: 0.5915032679738562

weighted
Precision: 0.7465576669734539
Recall: 0.6444444444444445
F1 Score: 0.6732026143790851

start RF 100 FGSM
(465, 56) (465,)
Accuracy: 0.6387096774193548

macro
Precision: 0.476723759332455
Recall: 0.4855965214240421
F1 Score: 0.4687738004569688

weighted
Precision: 0.5753023455688252
Recall: 0.6387096774193548
F1 Score: 0.597356460211778

start RF 100 PGD
(225, 56) (225,)
Accuracy: 0.6444444444444445

macro
Precision: 0.601510475881111
Recall: 0.6428571428571428
F1 Score: 0.591503267973

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9482758620689655

macro
Precision: 0.943288084464555
Recall: 0.9531298180696886
F1 Score: 0.9469996954005483

weighted
Precision: 0.9511260207000573
Recall: 0.9482758620689655
F1 Score: 0.9485594546619471

start RF 20 ZOO
(72, 56) (72,)
Accuracy: 0.9583333333333334

macro
Precision: 0.9444444444444444
Recall: 0.96875
F1 Score: 0.9544592030360531

weighted
Precision: 0.9629629629629629
Recall: 0.9583333333333334
F1 Score: 0.9588867805186589

start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input20/x_test_adv_CaFA_Def5.npy not found
start RF 20 SINIFGSM
(143, 56) (143,)
Accuracy: 0.5314685314685315

macro
Precision: 0.6465651834504293
Recall: 0.5745039682539682
F1 Score: 0.48723575060208724

weighted
Precision: 0.6659378872493626
Recall: 0.5314685314685315
F1 Score: 0.4693320059656694

start RF 20 VNIFGSM
(340, 56) (340,)
Accuracy: 0.4823529411764706

macro
Precision: 0.5045325045325045
Recall: 0.5024305555555556
F1

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.6638297872340425

macro
Precision: 0.33191489361702126
Recall: 0.5
F1 Score: 0.3989769820971867

weighted
Precision: 0.4406699864191942
Recall: 0.6638297872340425
F1 Score: 0.5297056102737117

start RF 1 PGD
(91, 56) (91,)
Accuracy: 0.3516483516483517

macro
Precision: 0.3505802707930368
Recall: 0.3502906976744186
F1 Score: 0.3503932244404114

weighted
Precision: 0.3523604055518949
Recall: 0.3516483516483517
F1 Score: 0.3519621334503367

start RF 1 DF
(17, 56) (17,)
Accuracy: 0.7647058823529411

macro
Precision: 0.38235294117647056
Recall: 0.5
F1 Score: 0.43333333333333335

weighted
Precision: 0.5847750865051903
Recall: 0.7647058823529411
F1 Score: 0.6627450980392158

start RF 1 AutoPGD
(62, 56) (62,)
Accuracy: 0.45161290322580644

macro
Precision: 0.4042328042328043
Recall: 0.310272536687631
F1 Score: 0.3345959595959596

weighted
Precision: 0.6648233486943165
Recall: 0.45161290322580644
F1 Score: 0.532624633431085

start RF 1 ZOO
(9, 56) (9,)
Accuracy: 1.0

macro
Precision

In [15]:
epsilon = 0

Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9642857142857143

macro
Precision: 0.975
Recall: 0.9444444444444444
F1 Score: 0.9577677224736048

weighted
Precision: 0.9660714285714286
Recall: 0.9642857142857143
F1 Score: 0.9636931695755225

start BIM
Accuracy: 0.5029585798816568

macro
Precision: 0.5107969244035643
Recall: 0.5106164988517929
F1 Score: 0.5024315869719513

weighted
Precision: 0.5154418693054353
Recall: 0.5029585798816568
F1 Score: 0.5008985166891716

start FGSM
Accuracy: 0.4677871148459384

macro
Precision: 0.510509190044851
Recall: 0.5076700898587934
F1 Score: 0.44713419405320814

weighted
Precision: 0.5223640067230833
Recall: 0.4677871148459384
F1 Score: 0.4312703563428501

start PGD
Accuracy: 0.5029585798816568

macro
Precision: 0.5107969244035643
Recall: 0.5106164988517929
F1 Score: 0.5024315869719513

weighted
Precision: 0.5154418693054353
Recall: 0.5029585798816568
F1 Score: 0.5008985166891716

start DF
Accuracy: 0.6181015452538632

macro
Precision: 0.6877251672671127
Recall: 0.662124

In [14]:

# torch.save(model.state_dict(), "./dnn_pytorch_FeatureSqueezing.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")